In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.optimizers import RMSprop
import os
import matplotlib
import matplotlib.pyplot as pl
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional
from hyperopt import Trials, STATUS_OK, tpe

/Users/peternagy/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/peternagy/anaconda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
'''
I have created this notebook to observe the task with binary classification. This way it was easier to find the most 
promising features and algorithm to go with.
I have found this (http://cs229.stanford.edu/proj2015/328_report.pdf) "paper" claiming that they have reached ~0.69
accuracy with RandomForest on the same dataset.
I couldn't reproduce their result, not even with their best features. More suprisingly I didn't find their best features the best at all.
With a fully connected Neural Network on binary classification my best result was ~0.63 accuracy.
'''

def data():
    cols = [' title_sentiment_polarity', ' n_tokens_title', ' data_channel_is_lifestyle', ' data_channel_is_entertainment', ' data_channel_is_bus', ' data_channel_is_socmed', ' data_channel_is_tech', ' data_channel_is_world', ' weekday_is_monday', ' weekday_is_tuesday', ' weekday_is_wednesday', ' weekday_is_thursday', ' weekday_is_friday', ' weekday_is_saturday', ' weekday_is_sunday', ' is_weekend', ' global_sentiment_polarity', ' shares']
    raw_data = pd.read_csv('data/OnlineNewsPopularity.csv', usecols=cols)
    
    def categorizeBinaryShares(shares):
        if shares <= 1400:
            return 0
        else:
            return 1
    
    raw_data['label'] = raw_data[' shares'].apply(categorizeBinaryShares)
    raw_data['label'].value_counts()
    raw_data.drop([' shares'], axis=1, inplace=True)
    
    y = raw_data['label']
    raw_data.drop(['label'], axis=1, inplace=True)
    
    raw_data.fillna("", inplace=True)
    raw_data = raw_data.reindex()
    
    def constructBestFeatures(df):
        from sklearn.tree import DecisionTreeClassifier
        if 'url' in raw_data.keys():
            raw_data.drop(['url'], axis=1, inplace=True)


        classifier = DecisionTreeClassifier()
        classifier.fit(raw_data, y)


        importances = []
        for name, importance in zip(raw_data.columns, classifier.feature_importances_):
            importances.append((name, importance))

        num_cols = len(raw_data.columns)
        importances_sorted = sorted(importances, key=lambda x: x[1])
        importances_sorted
        cols = []
        for imp_tupl in importances_sorted[-14:]:
            cols.append(imp_tupl[0])
        return cols

    cols = constructBestFeatures(raw_data)
    print(cols)
    best_features = raw_data[cols]
    
    X_train, X_test, y_train, y_test = train_test_split(best_features, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

In [3]:
def create_model(X_train, X_test, y_train, y_test):
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=X_train.shape[1]))
    model.add(Dense({{choice([32,64, 128 ,256, 512, 1024])}}, activation='relu'))
    model.add(Dropout({{uniform(0, 1)}}))
    model.add(Dense({{choice([32,64, 128 ,256, 512, 1024])}}, activation='relu'))
    model.add(Dropout({{uniform(0, 1)}}))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer={{choice(['rmsprop', 'adam', 'sgd'])}},
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    model.fit(np.array(X_train), np.array(y_train), epochs=10, batch_size={{choice([32, 64, 128])}}, validation_data=(np.array(X_test), np.array(y_test)), callbacks=[EarlyStopping(monitor='val_loss', mode='auto')])
    score, acc = model.evaluate(np.array(X_test), np.array(y_test), verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='fully_connected_binary')
X_train, X_test, y_train, y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(np.array(X_test), np.array(y_test)))
print("Best performing model chosen hyper-parameters:")
print(best_run)

>>> Imports:
#coding=utf-8

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from keras.layers import Dense, Flatten, Dropout, Activation
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from keras.callbacks import EarlyStopping
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    from keras.optimizers import RMSprop
except:
    pass

try:
    import os
except:
    pass

try:
    import matplotlib
except:
    pass

try:
    import matplotlib.pyplot as pl
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform, conditional
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from sklearn.tree import DecisionTreeClassifier
except:
    pass

>>> Hyperas search s

31715/31715 [==============================] - 0s - loss: 0.7134 - acc: 0.5097 - val_loss: 0.6895 - val_acc: 0.5918
Epoch 2/10
31715/31715 [==============================] - 0s - loss: 0.6911 - acc: 0.5233 - val_loss: 0.6883 - val_acc: 0.5701
Epoch 3/10
31715/31715 [==============================] - 0s - loss: 0.6902 - acc: 0.5297 - val_loss: 0.6874 - val_acc: 0.5842
Epoch 4/10
31715/31715 [==============================] - 0s - loss: 0.6891 - acc: 0.5360 - val_loss: 0.6845 - val_acc: 0.5735
Epoch 5/10
31715/31715 [==============================] - 0s - loss: 0.6876 - acc: 0.5435 - val_loss: 0.6839 - val_acc: 0.5918
Epoch 6/10
31715/31715 [==============================] - 0s - loss: 0.6858 - acc: 0.5493 - val_loss: 0.6814 - val_acc: 0.5896
Epoch 7/10
31715/31715 [==============================] - 0s - loss: 0.6852 - acc: 0.5534 - val_loss: 0.6805 - val_acc: 0.5914
Epoch 8/10
31715/31715 [==============================] - 0s - loss: 0.6828 - acc: 0.5638 - val_loss: 0.6777 - val_acc: 0.

In [4]:
#Save best model
def save_model(model,model_name, weights_name):
    model_json = model.to_json()
    with open(model_name, "w") as f:
        f.write(model_json)
    model.save_weights(weights_name)
    print("Model saved")

In [5]:
save_model(best_model, 'models/binary_model.json', 'models/binary_weights.h5')

Model saved


In [6]:
best_model.predict(np.array(X_train[:1]))

array([[0.45090124]], dtype=float32)